In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *

%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                'rnd_padding', 'aug_padding', "zoom_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/'
column = "EC number"
task = "task2/"

n_folds = 10

dicti = creating_dict()
dict_size = len(dicti)
max_lenn = 1000
n_class = 7 #number of classes to output
batch_size = 54
epochss = 200
callbacks_list = [None, None, None, None, False, False, False, False]
final_act = "softmax"
drop_per = 0.2 #Input dropout 
drop_hid = 0.5
n_neur = [313,76]


##### Each one of the following groups of parameters correspond to one of the architectures.
##### They must be uncommented according to the studied architecture in each case

############### only_denses architecture
architecture = "only_denses"
folder_arch = 'EC_number/archaea/3denses/bio_neurons/'
pool_size = None
n_filt = None
kernel_size = None
model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)

############## 1_conv architecture
#architecture = "conv_dense"
#folder_arch = 'EC_number/archaea/1conv/64filts_size5/'
#kernel_size = [5]
#n_filt = 64
#pool_size = None
#model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, 
                                        #drop_hid, n_filt, kernel_size, final_act, folder)

############## stack_conv architecture
#architecture = "stack_conv"
#folder_arch = 'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#kernel_size = [1,3,5,9,15]
#n_filt = 10
#pool_size = 10
#model = model = building_stackconv_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
#                                       kernel_size, pool_size, final_act, folder, optimizer)

model.summary()

In [3]:
lr=.00001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [5]:
#Building generators
generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 10)          1310      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 10)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 10)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               546816    
_________________________________________________________________
dense_1 (Dense)              (None, 313)               160569    
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder_arch, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 10)          1310      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 100, 10)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 10)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               546816    
_________________________________________________________________
dense_4 (Dense)              (None, 313)               160569    
__________


Epoch 00077: val_acc did not improve from 0.10026
Epoch 78/200
138/138 [==============================] - 41s 296ms/step - loss: 0.7956 - acc: 0.1735 - val_loss: 0.3102 - val_acc: 0.1003

Epoch 00078: val_acc did not improve from 0.10026
Epoch 79/200
138/138 [==============================] - 41s 295ms/step - loss: 0.7918 - acc: 0.1747 - val_loss: 0.3099 - val_acc: 0.1003

Epoch 00079: val_acc did not improve from 0.10026
Epoch 80/200
138/138 [==============================] - 41s 296ms/step - loss: 0.7949 - acc: 0.1727 - val_loss: 0.3095 - val_acc: 0.1003

Epoch 00080: val_acc did not improve from 0.10026
Epoch 81/200
138/138 [==============================] - 41s 295ms/step - loss: 0.7897 - acc: 0.1763 - val_loss: 0.3095 - val_acc: 0.1003

Epoch 00081: val_acc did not improve from 0.10026
Epoch 82/200
138/138 [==============================] - 41s 295ms/step - loss: 0.7926 - acc: 0.1769 - val_loss: 0.3094 - val_acc: 0.1003

Epoch 00082: val_acc did not improve from 0.10026
Epoch 83/

138/138 [==============================] - 41s 294ms/step - loss: 0.7830 - acc: 0.1767 - val_loss: 0.3025 - val_acc: 0.1003

Epoch 00164: val_acc did not improve from 0.10026
Epoch 165/200
138/138 [==============================] - 41s 296ms/step - loss: 0.7863 - acc: 0.1783 - val_loss: 0.3027 - val_acc: 0.1003

Epoch 00165: val_acc did not improve from 0.10026
Epoch 166/200
138/138 [==============================] - 41s 296ms/step - loss: 0.7845 - acc: 0.1757 - val_loss: 0.3032 - val_acc: 0.1003

Epoch 00166: val_acc did not improve from 0.10026
Epoch 167/200
138/138 [==============================] - 41s 294ms/step - loss: 0.7813 - acc: 0.1781 - val_loss: 0.3026 - val_acc: 0.1003

Epoch 00167: val_acc did not improve from 0.10026
Epoch 168/200
138/138 [==============================] - 41s 295ms/step - loss: 0.7795 - acc: 0.1757 - val_loss: 0.3022 - val_acc: 0.1003

Epoch 00168: val_acc did not improve from 0.10026
Epoch 169/200
138/138 [==============================] - 41s 295ms/st

Epoch 1/200
138/138 [==============================] - 43s 309ms/step - loss: 0.8747 - acc: 0.1946 - val_loss: 0.3583 - val_acc: 0.1003

Epoch 00001: val_acc improved from -inf to 0.10026, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/rnn_conv/256rnn/task2/post_padding/1/weights-improvement-001-0.1003.hdf5
Epoch 2/200
138/138 [==============================] - 41s 294ms/step - loss: 0.8414 - acc: 0.1679 - val_loss: 0.3372 - val_acc: 0.1003

Epoch 00002: val_acc did not improve from 0.10026
Epoch 3/200
138/138 [==============================] - 41s 294ms/step - loss: 0.8299 - acc: 0.1685 - val_loss: 0.3272 - val_acc: 0.1003

Epoch 00003: val_acc did not improve from 0.10026
Epoch 4/200
138/138 [==============================] - 41s 294ms/step - loss: 0.8238 - acc: 0.1707 - val_loss: 0.3215 - val_acc: 0.1003

Epoch 00004: val_acc did not improve from 0.10026
Epoch 5/200
138/138 [==============================] - 41s 295ms/step - loss: 0.8202 - acc: 0.1716 - v

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

138/138 [==============================] - 41s 297ms/step - loss: 0.8120 - acc: 0.1719 - val_loss: 0.3178 - val_acc: 0.1003

Epoch 00008: val_acc did not improve from 0.10026
Epoch 9/200
138/138 [==============================] - 41s 299ms/step - loss: 0.8065 - acc: 0.1708 - val_loss: 0.3161 - val_acc: 0.1003

Epoch 00009: val_acc did not improve from 0.10026
Epoch 10/200
138/138 [==============================] - 41s 297ms/step - loss: 0.8090 - acc: 0.1699 - val_loss: 0.3160 - val_acc: 0.1003

Epoch 00010: val_acc did not improve from 0.10026
Epoch 11/200
138/138 [==============================] - 41s 297ms/step - loss: 0.8068 - acc: 0.1714 - val_loss: 0.3158 - val_acc: 0.1003

Epoch 00011: val_acc did not improve from 0.10026
Epoch 12/200
138/138 [==============================] - 41s 296ms/step - loss: 0.8081 - acc: 0.1708 - val_loss: 0.3158 - val_acc: 0.1003

Epoch 00012: val_acc did not improve from 0.10026
Epoch 13/200
138/138 [==============================] - 41s 297ms/step - l


Epoch 00051: val_acc did not improve from 0.10026
Epoch 52/200
138/138 [==============================] - 41s 298ms/step - loss: 0.7972 - acc: 0.1736 - val_loss: 0.3133 - val_acc: 0.1003

Epoch 00052: val_acc did not improve from 0.10026
Epoch 53/200
138/138 [==============================] - 41s 298ms/step - loss: 0.7952 - acc: 0.1738 - val_loss: 0.3133 - val_acc: 0.1003

Epoch 00053: val_acc did not improve from 0.10026
Epoch 54/200
138/138 [==============================] - 41s 298ms/step - loss: 0.7934 - acc: 0.1740 - val_loss: 0.3132 - val_acc: 0.1003

Epoch 00054: val_acc did not improve from 0.10026
Epoch 55/200
138/138 [==============================] - 41s 299ms/step - loss: 0.7959 - acc: 0.1714 - val_loss: 0.3133 - val_acc: 0.1003

Epoch 00055: val_acc did not improve from 0.10026
Epoch 56/200
138/138 [==============================] - 41s 298ms/step - loss: 0.7996 - acc: 0.1704 - val_loss: 0.3139 - val_acc: 0.1003

Epoch 00056: val_acc did not improve from 0.10026
Epoch 57/

138/138 [==============================] - 41s 297ms/step - loss: 0.7879 - acc: 0.1761 - val_loss: 0.3079 - val_acc: 0.1003

Epoch 00138: val_acc did not improve from 0.10026
Epoch 139/200
138/138 [==============================] - 41s 297ms/step - loss: 0.7881 - acc: 0.1765 - val_loss: 0.3082 - val_acc: 0.1003

Epoch 00139: val_acc did not improve from 0.10026
Epoch 140/200
138/138 [==============================] - 41s 297ms/step - loss: 0.7883 - acc: 0.1762 - val_loss: 0.3083 - val_acc: 0.1003

Epoch 00140: val_acc did not improve from 0.10026
Epoch 141/200
138/138 [==============================] - 41s 299ms/step - loss: 0.7846 - acc: 0.1765 - val_loss: 0.3077 - val_acc: 0.1003

Epoch 00141: val_acc did not improve from 0.10026
Epoch 142/200
138/138 [==============================] - 41s 298ms/step - loss: 0.7913 - acc: 0.1758 - val_loss: 0.3083 - val_acc: 0.1003

Epoch 00142: val_acc did not improve from 0.10026
Epoch 143/200
138/138 [==============================] - 41s 297ms/st

138/138 [==============================] - 41s 297ms/step - loss: 0.8115 - acc: 0.1735 - val_loss: 0.3208 - val_acc: 0.1003

Epoch 00014: val_acc did not improve from 0.10026
Epoch 15/200
138/138 [==============================] - 41s 298ms/step - loss: 0.8079 - acc: 0.1730 - val_loss: 0.3203 - val_acc: 0.1003

Epoch 00015: val_acc did not improve from 0.10026
Epoch 16/200
138/138 [==============================] - 41s 297ms/step - loss: 0.8079 - acc: 0.1781 - val_loss: 0.3200 - val_acc: 0.1003

Epoch 00016: val_acc did not improve from 0.10026
Epoch 17/200
138/138 [==============================] - 41s 297ms/step - loss: 0.8050 - acc: 0.1782 - val_loss: 0.3195 - val_acc: 0.1003

Epoch 00017: val_acc did not improve from 0.10026
Epoch 18/200
138/138 [==============================] - 41s 298ms/step - loss: 0.8080 - acc: 0.1742 - val_loss: 0.3196 - val_acc: 0.1003

Epoch 00018: val_acc did not improve from 0.10026
Epoch 19/200
138/138 [==============================] - 41s 299ms/step - 


Epoch 00057: val_acc did not improve from 0.10026
Epoch 58/200
138/138 [==============================] - 41s 297ms/step - loss: 0.7982 - acc: 0.1782 - val_loss: 0.3158 - val_acc: 0.1003

Epoch 00058: val_acc did not improve from 0.10026
Epoch 59/200
138/138 [==============================] - 41s 297ms/step - loss: 0.7996 - acc: 0.1740 - val_loss: 0.3160 - val_acc: 0.1003

Epoch 00059: val_acc did not improve from 0.10026
Epoch 60/200
138/138 [==============================] - 41s 297ms/step - loss: 0.7942 - acc: 0.1769 - val_loss: 0.3160 - val_acc: 0.1003

Epoch 00060: val_acc did not improve from 0.10026
Epoch 61/200
138/138 [==============================] - 41s 298ms/step - loss: 0.7990 - acc: 0.1765 - val_loss: 0.3161 - val_acc: 0.1003

Epoch 00061: val_acc did not improve from 0.10026
Epoch 62/200
138/138 [==============================] - 41s 299ms/step - loss: 0.7977 - acc: 0.1773 - val_loss: 0.3162 - val_acc: 0.1003

Epoch 00062: val_acc did not improve from 0.10026
Epoch 63/